In [1]:
# Import the needed libraries
from Bio.PDB import PDBParser, is_aa, Polypeptide
from Bio.Align.Applications import ClustalOmegaCommandline
import os

In [2]:
# Get the total number of PDB files in the PDB Data folder
files_count = len(os.listdir("PDB Data")) - 1
print("The total number of PDB files is: ", files_count)

The total number of PDB files is:  2373


In [5]:
# Define a function that will preprocess the PDB file
def preprocess_sequence(pdb_files):
    sequences = []
    non_standard_residue_count = 0
    missing_atom_count = 0
    
    for pdb_file in pdb_files:
        structure = PDBParser().get_structure(pdb_file, f"PDB Data/{pdb_file}.pdb")
        
        for model in structure:
            for chain in model:
                sequence = ""
                for residue in chain:
                    # Check to see if it's not a standard amino acid
                    if not is_aa(residue):
                        non_standard_residue_count += 1
                        continue

                    # Ignore residues that are missing atoms  
                    if residue.is_disordered():
                        missing_atom_count += 1
                        continue
                    
                    # Convert to one-letter code
                    try:
                        sequence += Polypeptide.three_to_one(residue.get_resname())
                    except KeyError:
                        continue

                # Add correctly formed sequences
                if sequence:
                    sequences.append(sequence)
                    
    print(f"Found {non_standard_residue_count} non-standard residues")
    print(f"Found {missing_atom_count} missing atoms")
    
    return sequences

In [6]:
pdb_files_with_extension = os.listdir("PDB Data")
pdb_files = [file[:-4] for file in pdb_files_with_extension if file.endswith(".pdb")] 
sequences = preprocess_sequence(pdb_files)
print(sequences)

/home/jjk339/miniconda3/envs/SecondProject/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 8005.
  warnings.warn(
/home/jjk339/miniconda3/envs/SecondProject/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 8099.
  warnings.warn(
/home/jjk339/miniconda3/envs/SecondProject/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 8146.
  warnings.warn(
/home/jjk339/miniconda3/envs/SecondProject/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 8190.
  warnings.warn(
/home/jjk339/miniconda3/envs/SecondProject/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 8237.
  warnings.warn(
/home/jjk339/miniconda3/envs/Second

Found 457808 non-standard residues
Found 10056 missing atoms
['LALSLTADQMVSALLDAEPPILYSEYDPTRPFSEASMMGLLTNLADRELVHMINWAKRVPGFVDLTSHDQVHLLEAWLEILMIGLVWRSMEHPGKLLFAPNLLLDRNQGKCVEMVEIFDMLLATSSRFRMMNLQGEEFVCLKSIILLNSGVYTFLSSTLKSLEEKDHIHRVLDKITDTLIHLMAKAGLTLQQQHQRLAQLLLILSHIRHMSNKGMEHLYSVPSYDLLLEMLDA', 'SLALSLTADQMVSALLDAEPPILYSEYDPTRPSMMGLLTNLADRELVHMINWAKRVPGFVDLTSHDQVHLLEAWLEILMIGLVWRSMEHPGKLLFAPNLLLDRNQGMVEIFDMLLATSSRFRMMNLQGEEFVCLKSIILLNSGVYTFLSSTLKSLEEKDHIHRVLDKITDTLIHLMAKAGLTLQQQQRLAQLLLILSHIRHMSNKGMEHLYSMVVPSYDLLLEMLDA', 'LALSLTADQMVSALLDAEPPILYSEYDPTRPFSEASMMGLLTNLADRELVHMINWAKRVPGFVDLTSHDQVHLLEAWLEILMIGLVWRSMHPGKLLFAPNLLLDRNQGKVEIFDMLLATSSRFRMMNLQGEEFVCLKSIILLNSGVYTFLSSTLKSLEEKDHIHRVLDKITDTLIHLMAKAGLTLQQQHQRLAQLLLILSHIRHMSNKGMEHLVVPSYDLLLEMLDA', 'SLALSLTADQMVSALLDAEPPILYSEYDSMMGLLTNLADRELVHMINWAKRVPGFVDLTSHDQVHLLEAWLEILMIGLVWRSMEHPGKLLFAPNLLLDRNQGMVEIFDMLLATSSRFRMMNLQGEEFVCLKSIILLNSGVYTFKSLEEKDHIRVLDKITDTLIHLMAKAGLTLQQQHQRLAQLLLILSHIRHMSNKGMEHLYVVPSYDLLLEMLDA', 'ALYEDPPDQKTSPSGKPA

/home/jjk339/miniconda3/envs/SecondProject/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 44971.
  warnings.warn(
/home/jjk339/miniconda3/envs/SecondProject/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 44972.
  warnings.warn(
/home/jjk339/miniconda3/envs/SecondProject/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 44973.
  warnings.warn(
/home/jjk339/miniconda3/envs/SecondProject/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 44974.
  warnings.warn(
/home/jjk339/miniconda3/envs/SecondProject/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 44975.
  warnings.warn(
/home/jjk339/miniconda3/envs/S

In [9]:
print(len(sequences))

9479
